# Step 1 loading data

In [35]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch


In [36]:
path = '../data/data_clean.pkl'
train_data_unlabeled = pd.read_pickle(path)

path = '../data/catalogue_clean.pkl'
cat = pd.read_pickle(path)

# Step 2 generalistic function for adding a coumn with emeddings


In [37]:
from tqdm import tqdm
import numpy as np

def embed_text_column(df, text_column, model, target_column, batch_size=32):
    """
    Berechnet SentenceTransformer-Embeddings spaltenweise (batchweise, CPU-optimiert).
    """
    texts = df[text_column].fillna("").tolist()
    all_embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc=f"Embedding {text_column}"):
        batch = texts[i:i+batch_size]
        with torch.no_grad():
            emb = model.encode(batch, convert_to_tensor=True)
        all_embeddings.extend(emb.cpu().numpy())

    df[target_column] = all_embeddings
    return df



# Step 3 gespeicherte Models laden

In [ ]:
model_sbert_bert = SentenceTransformer("../models/raw_STM/base_bert_sentence_cls_embs")
model_jinai = SentenceTransformer("../models/raw_STM/base_jinai_sentence_mean_embs")
model_paraphrase = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Some weights of the model checkpoint at ../models/raw_STM/base_jinai_sentence_mean_embs/ were not used when initializing BertModel: ['encoder.layer.9.mlp.wo.bias', 'encoder.layer.2.mlp.layernorm.bias', 'encoder.layer.6.mlp.wo.weight', 'encoder.layer.10.mlp.gated_layers.weight', 'encoder.layer.8.mlp.wo.bias', 'encoder.layer.7.mlp.layernorm.bias', 'encoder.layer.8.mlp.gated_layers.weight', 'encoder.layer.5.mlp.layernorm.weight', 'encoder.layer.1.mlp.gated_layers.weight', 'encoder.layer.9.mlp.layernorm.bias', 'encoder.layer.1.mlp.layernorm.weight', 'encoder.layer.10.mlp.wo.weight', 'encoder.layer.3.mlp.gated_layers.weight', 'encoder.layer.0.mlp.layernorm.weight', 'encoder.layer.11.mlp.gated_layers.weight', 'encoder.layer.11.mlp.wo.bias', 'encoder.layer.7.mlp.wo.weight', 'encoder.layer.2.mlp.gated_layers.weight', 'encoder.layer.2.mlp.wo.bias', 'encoder.layer.5.mlp.gated_layers.weight', 'encoder.layer.0.mlp.wo.weight', 'encoder.layer.5.mlp.wo.bias', 'encoder.layer.10.mlp.layernorm.weight', 

In [ ]:
model_selected = model_paraphrase

In [ ]:
display(train_data_unlabeled.head())

,contract,paragraph,paragraph_title,paragraph_content,section,section_content,clean_paragraph_content,clean_section_content,paragraph_content_stemm,paragraph_content_lemma,paragraph_content_token,section_content_stemm,paragraph_section_lemma,paragraph_section_token
0,1,§ 1,Vertragsgegenstand,§ 1 Vertragsgegenstand 1.1 Der Anbieter stellt...,1.1,1.1 Der Anbieter stellt dem Kunden die Softwar...,Der Anbieter stellt dem Kunden die Software Na...,Der Anbieter stellt dem Kunden die Software Na...,der anbiet stellt dem kunden die softwar name ...,Der Anbieter stellt dem Kunden die Software Na...,"[der, an, ##bie, ##ter, ste, ##ll, ##t, dem, k...",der anbiet stellt dem kunden die softwar name ...,Der Anbieter stellt dem Kunden die Software Na...,"[der, an, ##bie, ##ter, ste, ##ll, ##t, dem, k..."
1,1,§ 1,Vertragsgegenstand,§ 1 Vertragsgegenstand 1.1 Der Anbieter stellt...,1.2,1.2 Die Nutzung umfasst die Bereitstellung von...,Der Anbieter stellt dem Kunden die Software Na...,Die Nutzung umfasst die Bereitstellung von Fun...,der anbiet stellt dem kunden die softwar name ...,Der Anbieter stellt dem Kunden die Software Na...,"[der, an, ##bie, ##ter, ste, ##ll, ##t, dem, k...",die nutzung umfasst die bereitstellung von fun...,die Nutzung umfasst die Bereitstellung von Fun...,"[die, nut, ##zu, ##ng, um, ##fa, ##ss, ##t, di..."
2,1,§ 1,Vertragsgegenstand,§ 1 Vertragsgegenstand 1.1 Der Anbieter stellt...,1.3,1.3 Der Kunde erhält ausschließlich das vertra...,Der Anbieter stellt dem Kunden die Software Na...,Der Kunde erhält ausschließlich das vertraglic...,der anbiet stellt dem kunden die softwar name ...,Der Anbieter stellt dem Kunden die Software Na...,"[der, an, ##bie, ##ter, ste, ##ll, ##t, dem, k...",der kund erhält ausschließlich da vertraglich ...,Der Kunde erhält ausschließlich das vertraglic...,"[der, kun, ##de, er, ##hal, ##t, aus, ##sch, #..."
3,2,§ 1,Vertragsgegenstand,§ 1 Vertragsgegenstand (1) Dieser Software-as-...,(1),(1) Dieser Software-as-a-Service-Vertrag ist a...,1 Dieser SoftwareasaServiceVertrag ist auf On ...,1 Dieser SoftwareasaServiceVertrag ist auf On ...,1 dieser softwareasaservicevertrag ist auf on ...,1 Dieser softwareasaservicevertrag ist auf on ...,"[1, dies, ##er, software, ##asa, ##ser, ##vic,...",1 dieser softwareasaservicevertrag ist auf on ...,1 Dieser softwareasaservicevertrag ist auf on ...,"[1, dies, ##er, software, ##asa, ##ser, ##vic,..."
4,2,§ 1,Vertragsgegenstand,§ 1 Vertragsgegenstand (1) Dieser Software-as-...,(2),(2) Die Software wird vom Anbieter als webbasi...,1 Dieser SoftwareasaServiceVertrag ist auf On ...,2 Die Software wird vom Anbieter als webbasier...,1 dieser softwareasaservicevertrag ist auf on ...,1 Dieser softwareasaservicevertrag ist auf on ...,"[1, dies, ##er, software, ##asa, ##ser, ##vic,...",2 die softwar wird vom anbiet al webbasiert sa...,2 die Software wird vom Anbieter al webbasiert...,"[2, die, software, wi, ##rd, vo, ##m, an, ##bi..."


In [ ]:
train_data_unlabeled = embed_text_column(train_data_unlabeled.iloc[0:100,:], text_column="clean_section_content",model = model_selected, target_column="section_emb_jinai" )
cat = embed_text_column(cat, text_column = "example", model = model_selected, target_column = "example_emb_jinai")

Embedding clean_section_content: 100%|██████████| 1/1 [00:14<00:00, 14.81s/it]
/var/folders/n9/9rvxcg4d1d15_3wh3wq9f0l40000gn/T/ipykernel_3402/1283774044.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_column] = all_embeddings
Embedding example: 100%|██████████| 3/3 [00:43<00:00, 14.34s/it]


In [ ]:
train_data_unlabeled = train_data_unlabeled[["contract","paragraph","section","section_content","section_emb_jinai"]]
display(train_data_unlabeled.head())
cat

,contract,paragraph,section,section_content,section_emb_jinai
211,7,7.,7.1,7.1 Der Kunde wird den Anbieter bei der Erbrin...,"[-0.04252922, -0.004851811, 0.037896913, -0.01..."
212,7,7.,7.2,7.2 Die ordnungsgemäße und regelmäßige Sicheru...,"[0.00532906, -0.07898556, 0.009738089, 0.04176..."
213,7,7.,7.3,7.3 Für die Nutzung der Software müssen die si...,"[-0.050004583, -0.023699032, 0.05629496, 0.016..."
214,7,7.,7.4,7.4 Der Kunde hat die ihm zur Verfügung gestel...,"[-0.010814101, -0.009543186, 0.023113329, 0.06..."
215,7,8.,no sections use paragraph,8. Gewährleistung Es gelten grundsätzlich die ...,"[-0.021406822, -0.06396903, 0.054216247, 0.037..."


,paragraph_topic,section_topic,example,example_emb_jinai
0,Projektkosten_und_Zahlungsmodalitäten,Sind sämtliche Kostenarten und -bestandteile (...,Im Festpreis von € sind sämtliche Leistungen e...,"[-0.03012688, -0.028163651, 0.015150636, -0.02..."
1,Projektkosten_und_Zahlungsmodalitäten,Ist das Vergütungsmodell eindeutig festgelegt ...,Der Kunde zahlt eine monatliche Pauschale von ...,"[0.013873805, -0.011931838, -0.0193584, 0.0172..."
2,Projektkosten_und_Zahlungsmodalitäten,Ist ein Zahlungsplan mit konkreten Fälligkeite...,Die Vergütung ist in drei Raten zahlbar 30 bei...,"[-0.0013125225, -0.02878987, 0.06904589, 0.029..."
3,Projektkosten_und_Zahlungsmodalitäten,"Sind Währung, Rechnungsstellung, Zahlungsfrist...",Alle Preise verstehen sich in Euro zuzüglich g...,"[-0.017860401, -0.031492785, 0.036198236, 0.02..."
4,Projektkosten_und_Zahlungsmodalitäten,Regelt der Vertrag den Umgang mit Nebenkosten ...,Reise und Übernachtungskosten werden nur ersta...,"[-0.058175255, -0.023304999, 0.014267244, 0.02..."
...,...,...,...,...
71,Sonstige_wichtige_Klauseln,Ist die anwendbare Rechtsordnung eindeutig ver...,Dieser Vertrag unterliegt dem Recht der Bundes...,"[-0.016920192, -0.031987045, 0.007448015, 0.00..."
72,Sonstige_wichtige_Klauseln,Ist ein Gerichtsstand für Streitigkeiten festg...,Gerichtsstand für alle Streitigkeiten aus oder...,"[-0.04890105, -0.015488585, 0.085540846, -0.01..."
73,Sonstige_wichtige_Klauseln,"Bei mehrsprachigen Verträgen: Ist festgelegt, ...",Dieser Vertrag wird in deutscher und englische...,"[-0.028063385, -0.006123979, 0.0443462, 0.0201..."
74,Sonstige_wichtige_Klauseln,Enthält der Vertrag eine salvatorische Klausel...,Sollte eine Bestimmung dieses Vertrages unwirk...,"[0.0033766574, -0.025280597, 0.0743445, 0.0105..."


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Matrizen aus Embedding-Listen bauen
X = np.vstack(train_data_unlabeled["section_emb_jinai"].values)  # Shape: [1400, 768]
Y = np.vstack(cat["example_emb_jinai"].values)                  # Shape: [100, 768]

# Cosine Similarity: alle Kombinationen
similarity_matrix = cosine_similarity(X, Y)  # Shape: [1400, 100]
similarity_percent = np.round(similarity_matrix * 100, 2)  # Skaliert zu 0–100 %



In [ ]:
best_match_idx = similarity_matrix.argmax(axis=1)
best_match_score = similarity_percent[np.arange(len(X)), best_match_idx]

# Ergebnisse anhängen
train_data_unlabeled["matched_example_index"] = best_match_idx
train_data_unlabeled["similarity_percent"] = best_match_score
train_data_unlabeled["matched_example_text"] = cat.loc[best_match_idx, "example"].values
train_data_unlabeled["matched_example_topic"] = cat.loc[best_match_idx, "section_topic"].values
train_data_unlabeled["matched_paragraph"] = cat.loc[best_match_idx, "paragraph_topic"].values

display(train_data_unlabeled.head())
print(train_data_unlabeled.iloc[3,3])
print(f"-->{train_data_unlabeled.iloc[3,7]}")
print(train_data_unlabeled["similarity_percent"].mean())


,contract,paragraph,section,section_content,section_emb_jinai,matched_example_index,similarity_percent,matched_example_text,matched_example_topic,matched_paragraph
211,7,7.,7.1,7.1 Der Kunde wird den Anbieter bei der Erbrin...,"[-0.04252922, -0.004851811, 0.037896913, -0.01...",3,63.310001,Alle Preise verstehen sich in Euro zuzüglich g...,"Sind Währung, Rechnungsstellung, Zahlungsfrist...",Projektkosten_und_Zahlungsmodalitäten
212,7,7.,7.2,7.2 Die ordnungsgemäße und regelmäßige Sicheru...,"[0.00532906, -0.07898556, 0.009738089, 0.04176...",24,66.190002,Verweigert der Kunde die Abnahme wegen erhebli...,"Ist geregelt, was passiert, wenn bei der Abnah...",Abnahmeprozesse
213,7,7.,7.3,7.3 Für die Nutzung der Software müssen die si...,"[-0.050004583, -0.023699032, 0.05629496, 0.016...",34,66.029999,Der Anbieter speichert und verarbeitet Kundend...,"Ist geregelt, wo die Kundendaten gespeichert/v...",Datenschutz_und_Informationssicherheit
214,7,7.,7.4,7.4 Der Kunde hat die ihm zur Verfügung gestel...,"[-0.010814101, -0.009543186, 0.023113329, 0.06...",64,74.300003,Treten während der Gewährleistungsfrist Mängel...,Sind die Rechte des Kunden im Gewährleistungsf...,Haftung_und_Gewährleistung
215,7,8.,no sections use paragraph,8. Gewährleistung Es gelten grundsätzlich die ...,"[-0.021406822, -0.06396903, 0.054216247, 0.037...",65,68.519997,HGB Untersuchungs und Rügepflicht findet auf d...,Wird die kaufmännische Untersuchungs- und Rüge...,Haftung_und_Gewährleistung


7.4 Der Kunde hat die ihm zur Verfügung gestellten Zugangsdaten geheim zu halten und dafür zu sorgen, dass etwaige Mitarbeiter, denen Zugangsdaten zur Verfügung gestellt werden, dies ebenfalls tun. Die Leistung des Anbieters darf Dritten nicht zur Verfügung gestellt werden, soweit das nicht von den Parteien ausdrücklich vereinbart wurde.
-->Treten während der Gewährleistungsfrist Mängel auf hat der Kunde diese dem Anbieter unverzüglich anzuzeigen Der Anbieter ist verpflichtet Sachmängel nach eigener Wahl durch Nachbesserung oder Austausch innerhalb angemessener Frist zu beheben Schlagen zwei Nacherfüllungsversuche fehl oder verzögert der Anbieter die Nacherfüllung unzumutbar kann der Kunde eine Herabsetzung der Vergütung Minderung verlangen oder – bei erheblichen Mängeln – vom Vertrag zurücktreten
